### Consulting Project - Recommender Systems


This company is trying to differentiate itself by recommending new meals to customers based off of other customers likings.

Can you build them a recommendation system?

Your final result should be in the form of a function that can take in a Spark DataFrame of a single customer's ratings for various meals and output their top 3 suggested meals. For example:



In [6]:
import findspark

findspark.init('/home/tosin/spark-3.0.1-bin-hadoop2.7')

from pyspark.sql import SparkSession

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

import pandas as pd

import numpy as np

In [7]:
#start spark session

spark = SparkSession.builder.appName('reccy').getOrCreate()

In [14]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [16]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [17]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [18]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [19]:
# Split dataset: Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [20]:
# Build the recommendation model using ALS on the training data

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

model = als.fit(training)

### Predict and Evaluate trained Model

In [21]:
# Evaluate the model by computing the RMSE on the test data

predictions = model.transform(test)

In [22]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    13| 0.83028555|
|     31|   3.0|     8| -0.5072875|
|     85|   1.0|    15| -2.6138585|
|     85|   1.0|    23|   1.224613|
|     85|   1.0|     2|  1.7797219|
|     65|   1.0|    28| -0.8468907|
|     65|   1.0|    19|  1.3386711|
|     53|   2.0|    19|  1.5497864|
|     53|   1.0|     9|  1.8938087|
|     53|   5.0|     8|  2.1211896|
|     78|   1.0|    12|  1.0652735|
|     78|   1.0|    11|  0.8280753|
|     34|   1.0|    19| 0.84316945|
|     34|   3.0|    25|   2.562931|
|     81|   1.0|     3|-0.10146451|
|     81|   1.0|    21| -1.1878756|
|     81|   4.0|    11|  1.6324582|
|     28|   1.0|     5|  1.2792455|
|     28|   1.0|    23|  2.5335968|
|     76|   1.0|    20| -1.0987935|
+-------+------+------+-----------+
only showing top 20 rows



In [23]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.9335184611087075


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [25]:
single_user = test.filter(test['userId']==11).select(['movieId','userId', 'rating'])

In [26]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|     10|    11|   1.0|
|     16|    11|   1.0|
|     22|    11|   1.0|
|     35|    11|   3.0|
|     36|    11|   2.0|
|     41|    11|   1.0|
|     50|    11|   4.0|
|     69|    11|   5.0|
|     70|    11|   1.0|
|     71|    11|   3.0|
|     72|    11|   3.0|
|     78|    11|   1.0|
|     80|    11|   3.0|
|     81|    11|   4.0|
|     86|    11|   1.0|
|     97|    11|   3.0|
+-------+------+------+



In [27]:
reccomendations = model.transform(single_user)

In [28]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------+----------+
|movieId|userId|rating|prediction|
+-------+------+------+----------+
|     22|    11|   1.0|  6.502684|
|     36|    11|   2.0|   5.08809|
|     10|    11|   1.0| 2.6902778|
|     71|    11|   3.0| 2.5280564|
|     80|    11|   3.0| 2.3568196|
|     41|    11|   1.0| 2.2901683|
|     16|    11|   1.0|  1.985836|
|     86|    11|   1.0| 1.9659646|
|     50|    11|   4.0|   1.71031|
|     81|    11|   4.0| 1.6324582|
|     35|    11|   3.0| 1.5472388|
|     70|    11|   1.0| 1.1085926|
|     97|    11|   3.0| 0.8526864|
|     78|    11|   1.0| 0.8280753|
|     72|    11|   3.0|0.59547985|
|     69|    11|   5.0|0.17478818|
+-------+------+------+----------+

